# Metrics

https://towardsdatascience.com/the-5-classification-evaluation-metrics-you-must-know-aa97784ff226

https://towardsdatascience.com/20-popular-machine-learning-metrics-part-1-classification-regression-evaluation-metrics-1ca3e282a2ce

In [44]:
import numpy as np
import json
import copy
from json import JSONEncoder
import os

In [45]:
class EvaluatedModel:
    y_pred = []
    y_true = []
    model_name = ''
    avg_time = 0.0
    gzip_size = 0
    
    accuracy = None
    precision = None
    recall = None
    f1_score = None
    

    def __init__(self, dict1=None, y_pred=[], y_true=[], model_name='', avg_time=0.0, gzip_size=0):
        if (dict1 == None):
            self.y_pred = copy.deepcopy(y_pred)
            self.y_true = copy.deepcopy(y_true)
            self.model_name = model_name
            self.avg_time = avg_time
            self.gzip_size = gzip_size
        else:
            self.__dict__.update(dict1)


## 1. Compute metrics for models

In [46]:
from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score 
from sklearn.metrics import f1_score

def annote_models(path):
    with open(path) as f:
        evaluated_models = json.load(f, object_hook=EvaluatedModel)

    for evaluated_model in evaluated_models:
        evaluated_model.accuracy = accuracy_score(y_pred=evaluated_model.y_pred, y_true=evaluated_model.y_true)
        evaluated_model.precision = precision_score(y_pred=evaluated_model.y_pred, y_true=evaluated_model.y_true, average='macro')
        evaluated_model.recall = recall_score(y_pred=evaluated_model.y_pred, y_true=evaluated_model.y_true, average='macro')
        evaluated_model.f1_score = f1_score(y_pred=evaluated_model.y_pred, y_true=evaluated_model.y_true, average='macro')
        # change model name for table
        evaluated_model.model_name = evaluated_model.model_name.replace("Efficent", "Efficient")
        evaluated_model.model_name = evaluated_model.model_name.replace("EfficientNetB0", "EffiNet")
        evaluated_model.model_name = evaluated_model.model_name.replace("MobileNetV2", "MobNet")
        evaluated_model.model_name = evaluated_model.model_name.replace("KMeansPlusPlus32", "WC32")
        evaluated_model.model_name = evaluated_model.model_name.replace("KMeansPlusPlus128", "WC128")
        evaluated_model.model_name = evaluated_model.model_name.replace("full_integer_quantization", "FIQ")
        evaluated_model.model_name = evaluated_model.model_name.replace("dynamic_rage_quantization", "DRQ")
        evaluated_model.model_name = evaluated_model.model_name.replace("float16_quantization", "F16Q")
        evaluated_model.model_name = evaluated_model.model_name.replace("_integer_io", "-Iio")
        evaluated_model.model_name = evaluated_model.model_name.replace("PolynomialDecay90", "PD90")
        evaluated_model.model_name = evaluated_model.model_name.replace("PolynomialDecay75", "PD75")
        evaluated_model.model_name = evaluated_model.model_name.replace("PolynomialDecay50", "PD50")
        evaluated_model.model_name = evaluated_model.model_name.replace("flowers_model_", "")
        evaluated_model.model_name = evaluated_model.model_name.replace("beans_model_", "")
        evaluated_model.model_name = evaluated_model.model_name.replace("_flowers_model", "")
        evaluated_model.model_name = evaluated_model.model_name.replace("_beans_model", "")
        evaluated_model.model_name = evaluated_model.model_name.replace("_tf", "")
        evaluated_model.model_name = evaluated_model.model_name.replace("_tf", "")
        evaluated_model.model_name = evaluated_model.model_name.replace("ConstantSparsity90", "CS90")
        evaluated_model.model_name = evaluated_model.model_name.replace("ConstantSparsity75", "CS75")
        evaluated_model.model_name = evaluated_model.model_name.replace("ConstantSparsity50", "CS50")
        evaluated_model.model_name = evaluated_model.model_name.replace(".tflite", "")
        evaluated_model.model_name = evaluated_model.model_name.replace("_", " -> ")
    evaluated_models.sort(key=lambda x: x.model_name, reverse=False)
    return evaluated_models

def print_table(models):
    row_counter = 0
    for model in models:
        if row_counter % 45 == 0:
            print('\n')
            print("|Model name|Accuarcy|Precision|Recall|F1 score|CMS|Average time|")
            print("|-----|-----|-----|-----|-----|-----|-----|")
        print("|"+model.model_name+"|"+str(round(model.accuracy * 100, 2))+" \%|"+str(round(model.precision * 100, 2))
              +" \%|"+str(round(model.recall * 100, 2))+" \%|"+str(round(model.f1_score * 100, 2))+" \%|"
              +str(round(model.gzip_size / 1024))+" kB|"+str(round(model.avg_time, 2))+" ms|")
        row_counter+=1


# 1. beans dataset

## 1.1 MobileNetV2

In [47]:
flowers_models = annote_models('results/evaluated_beans_models_on_MobileNetV2.json')
print_table(flowers_models)



|Model name|Accuarcy|Precision|Recall|F1 score|CMS|Average time|
|-----|-----|-----|-----|-----|-----|-----|
|MobNet|70.31 \%|72.89 \%|70.41 \%|70.38 \%|7993 kB|230.08 ms|
|MobNet -> CS50|52.34 \%|68.81 \%|52.68 \%|49.75 \%|8106 kB|230.85 ms|
|MobNet -> CS50 -> DRQ|53.12 \%|69.51 \%|53.45 \%|50.76 \%|2220 kB|335.23 ms|
|MobNet -> CS50 -> F16Q|53.12 \%|69.51 \%|53.45 \%|50.76 \%|4052 kB|270.1 ms|
|MobNet -> CS50 -> FIQ|52.34 \%|68.4 \%|52.69 \%|49.19 \%|2252 kB|253.78 ms|
|MobNet -> CS50 -> FIQ-Iio|53.12 \%|68.45 \%|53.45 \%|50.53 \%|2252 kB|256.27 ms|
|MobNet -> CS50 -> WC128|33.59 \%|11.2 \%|33.33 \%|16.76 \%|4894 kB|241.25 ms|
|MobNet -> CS50 -> WC128 -> DRQ|33.59 \%|11.2 \%|33.33 \%|16.76 \%|2142 kB|326.78 ms|
|MobNet -> CS50 -> WC128 -> F16Q|33.59 \%|11.2 \%|33.33 \%|16.76 \%|3734 kB|241.66 ms|
|MobNet -> CS50 -> WC128 -> FIQ|33.59 \%|11.2 \%|33.33 \%|16.76 \%|2177 kB|251.11 ms|
|MobNet -> CS50 -> WC128 -> FIQ-Iio|33.59 \%|11.2 \%|33.33 \%|16.76 \%|2177 kB|167.69 ms|
|MobNet -> C

## 1.2 EfficientNetB0

In [48]:
beans_models = annote_models('results/evaluated_beans_models_on_EfficentNetB0.json')
print_table(beans_models)



|Model name|Accuarcy|Precision|Recall|F1 score|CMS|Average time|
|-----|-----|-----|-----|-----|-----|-----|
|EffiNet|73.44 \%|75.0 \%|73.39 \%|73.58 \%|14488 kB|2613.81 ms|
|EffiNet -> DRQ|75.0 \%|76.35 \%|74.95 \%|75.16 \%|4226 kB|2400.73 ms|
|EffiNet -> F16Q|73.44 \%|75.0 \%|73.39 \%|73.58 \%|7220 kB|2556.4 ms|
|EffiNet -> FIQ|70.31 \%|72.97 \%|70.23 \%|70.52 \%|4286 kB|2034.77 ms|


# 2. TF Flowers

## 2.1 MobileNetV2

In [49]:
tf_flowers_models = annote_models('results/evaluated_tf_flowers_models_on_MobileNetV2.json')
print_table(tf_flowers_models)



|Model name|Accuarcy|Precision|Recall|F1 score|CMS|Average time|
|-----|-----|-----|-----|-----|-----|-----|
|MobNet|66.73 \%|70.28 \%|68.53 \%|67.22 \%|8463 kB|62.62 ms|
|MobNet -> CS50|37.09 \%|27.72 \%|33.94 \%|26.32 \%|8536 kB|62.97 ms|
|MobNet -> CS50 -> DRQ|36.73 \%|27.14 \%|33.64 \%|25.97 \%|2301 kB|75.75 ms|
|MobNet -> CS50 -> F16Q|36.91 \%|27.36 \%|33.79 \%|26.2 \%|4262 kB|62.81 ms|
|MobNet -> CS50 -> FIQ|36.73 \%|27.64 \%|33.6 \%|25.94 \%|2334 kB|46.49 ms|
|MobNet -> CS50 -> FIQ-Iio|36.36 \%|27.15 \%|33.21 \%|25.58 \%|2334 kB|47.22 ms|
|MobNet -> CS50 -> WC128|73.09 \%|74.36 \%|72.8 \%|72.87 \%|5055 kB|64.25 ms|
|MobNet -> CS50 -> WC128 -> DRQ|72.91 \%|74.18 \%|72.66 \%|72.61 \%|2221 kB|77.02 ms|
|MobNet -> CS50 -> WC128 -> F16Q|73.09 \%|74.36 \%|72.8 \%|72.87 \%|3967 kB|62.49 ms|
|MobNet -> CS50 -> WC128 -> FIQ|72.73 \%|74.02 \%|72.56 \%|72.45 \%|2254 kB|54.95 ms|
|MobNet -> CS50 -> WC128 -> FIQ-Iio|72.55 \%|73.81 \%|72.27 \%|72.18 \%|2254 kB|47.21 ms|
|MobNet -> CS50 -> W

## 2.2 EfficientNetB0

In [50]:
tf_flowers_models = annote_models('results/evaluated_tf_flowers_models_on_EfficentNetB0.json')
print_table(tf_flowers_models)



|Model name|Accuarcy|Precision|Recall|F1 score|CMS|Average time|
|-----|-----|-----|-----|-----|-----|-----|
|EffiNet|70.91 \%|71.36 \%|71.11 \%|70.85 \%|14923 kB|652.35 ms|
|EffiNet -> CS50|65.45 \%|68.11 \%|64.4 \%|64.74 \%|15072 kB|606.35 ms|
|EffiNet -> CS50 -> DRQ|65.64 \%|67.73 \%|64.66 \%|64.99 \%|4062 kB|674.65 ms|
|EffiNet -> CS50 -> F16Q|65.64 \%|68.31 \%|64.58 \%|64.92 \%|7540 kB|637.01 ms|
|EffiNet -> CS50 -> FIQ|66.36 \%|69.11 \%|65.29 \%|65.71 \%|4122 kB|559.7 ms|
|EffiNet -> CS50 -> WC128|73.27 \%|73.46 \%|73.55 \%|73.06 \%|8625 kB|592.71 ms|
|EffiNet -> CS50 -> WC128 -> DRQ|72.36 \%|72.43 \%|72.73 \%|72.23 \%|3848 kB|699.26 ms|
|EffiNet -> CS50 -> WC128 -> F16Q|73.27 \%|73.46 \%|73.55 \%|73.06 \%|6715 kB|562.34 ms|
|EffiNet -> CS50 -> WC128 -> FIQ|72.73 \%|72.84 \%|72.98 \%|72.54 \%|3901 kB|532.7 ms|
|EffiNet -> CS50 -> WC32|73.82 \%|74.41 \%|74.62 \%|74.14 \%|5980 kB|662.45 ms|
|EffiNet -> CS50 -> WC32 -> DRQ|74.0 \%|74.4 \%|74.76 \%|74.27 \%|3288 kB|714.93 ms|
|Effi

# 3. Oxford Flowers

## 3.1 MobileNetV2

In [51]:
flowers_models = annote_models('results/evaluated_flowers_models_on_MobileNetV2.json')
print_table(flowers_models)

FileNotFoundError: [Errno 2] No such file or directory: 'results/evaluated_flowers_models_on_MobileNetV2.json'

## 3.2 EfficientNetB0

In [41]:
flowers_models = annote_models('results/evaluated_flowers_models_on_EfficentNetB0.json')
print_table(flowers_models)

FileNotFoundError: [Errno 2] No such file or directory: 'results/evaluated_flowers_models_on_EfficentNetB0.json'